In [1]:
import numpy as np
import pandas as pd
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

# Function to load and prepare data
def load_and_prepare_data(filepath, time_column, target_column):
    """
    Load and prepare the dataset for Prophet.
    """
    data = pd.read_csv(filepath)
    data.rename(columns={time_column: "ds", target_column: "y"}, inplace=True)
    data["ds"] = pd.to_datetime(data["ds"])
    data.sort_values(by="ds", inplace=True)
    return data

# Function for expanding window forecast
def prophet_expanding_window_forecast(data, train_size):
    """
    Perform expanding window forecast using Prophet.
    """
    test_size = len(data) - train_size
    actuals, predictions, lower_bounds, upper_bounds = [], [], [], []

    for i in range(test_size):
        # Split the data into training and testing sets
        train = data.iloc[: train_size + i]
        test = data.iloc[train_size + i : train_size + i + 1]  # Forecast one step ahead

        # Initialize and fit the Prophet model
        model = Prophet()
        model.fit(train)

        # Create a DataFrame for the next step's prediction
        future = model.make_future_dataframe(periods=1, freq="D")  # Forecasting one step ahead

        # Predict the next step
        forecast = model.predict(future)
        forecast_next = forecast.iloc[-1]  # Last row is the next step forecast

        # Save forecast results
        actuals.append(test["y"].values[0])
        predictions.append(forecast_next["yhat"])
        lower_bounds.append(forecast_next["yhat_lower"])
        upper_bounds.append(forecast_next["yhat_upper"])

    # Create a DataFrame for comparison
    comparison_df = pd.DataFrame({
        "Actual": actuals,
        "Predicted": predictions,
        "Lower_CI": lower_bounds,
        "Upper_CI": upper_bounds
    }, index=data["ds"][train_size:])
    
    return comparison_df

# Function to plot the results
def plot_forecast(data, train_size, comparison_df):
    """
    Plot the actuals and forecasted values along with confidence intervals.
    """
    plt.figure(figsize=(14, 7))

    # Plot training data
    plt.plot(data["ds"][:train_size], data["y"][:train_size], label="Training Data", color="blue")

    # Plot actual test data
    plt.plot(data["ds"][train_size:], comparison_df["Actual"], label="Testing Data (Actual)", color="orange")

    # Plot predictions
    plt.plot(data["ds"][train_size:], comparison_df["Predicted"], label="Forecast", color="red", linestyle="--")

    # Shade the confidence intervals
    plt.fill_between(data["ds"][train_size:], comparison_df["Lower_CI"], comparison_df["Upper_CI"], 
                     color="green", alpha=0.3, label="Confidence Interval (Min-Max)")

    # Add titles and labels
    plt.title("FB Prophet Expanding Window Forecast vs Actuals with Confidence Intervals")
    plt.xlabel("Date")
    plt.ylabel("Target Value")
    plt.legend()
    plt.grid()

    # Show the plot
    plt.show()

# Main function to run the pipeline
def main(filepath, time_column, target_column, train_size):
    """
    Main pipeline to load data, perform Prophet forecasting, and evaluate results.
    """
    # Load and prepare data
    data = load_and_prepare_data(filepath, time_column, target_column)

    # Perform expanding window forecast
    comparison_df = prophet_expanding_window_forecast(data, train_size)

    # Display the comparison DataFrame
    print(comparison_df.head())

    # Plot the results
    plot_forecast(data, train_size, comparison_df)

    # Calculate evaluation metrics
    mae = mean_absolute_error(comparison_df["Actual"], comparison_df["Predicted"])
    mse = mean_squared_error(comparison_df["Actual"], comparison_df["Predicted"])
    rmse = np.sqrt(mse)

    # Print metrics
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")

    # Save results to a CSV
    output_file = "Forecast_results_Prophet_ExpandingWindow.csv"
    comparison_df.to_csv(output_file, index=False)
    print(f"Forecast results saved to '{output_file}'.")

# Run the pipeline
if __name__ == "__main__":
    # Inputs for dynamic code
    FILEPATH = r"C:\upi_data\transactions_single_account.csv"
    TIME_COLUMN = "transaction_date"
    TARGET_COLUMN = "total_revenue"
    TRAIN_SIZE = 15  # Initial training set size

    main(FILEPATH, TIME_COLUMN, TARGET_COLUMN, TRAIN_SIZE)


20:47:02 - cmdstanpy - INFO - Chain [1] start processing
20:47:03 - cmdstanpy - INFO - Chain [1] done processing
20:47:03 - cmdstanpy - INFO - Chain [1] start processing
20:47:03 - cmdstanpy - INFO - Chain [1] done processing
20:47:03 - cmdstanpy - INFO - Chain [1] start processing
20:47:03 - cmdstanpy - INFO - Chain [1] done processing
20:47:04 - cmdstanpy - INFO - Chain [1] start processing
20:47:04 - cmdstanpy - INFO - Chain [1] done processing
20:47:04 - cmdstanpy - INFO - Chain [1] start processing
20:47:04 - cmdstanpy - INFO - Chain [1] done processing
20:47:04 - cmdstanpy - INFO - Chain [1] start processing
20:47:04 - cmdstanpy - INFO - Chain [1] done processing
20:47:04 - cmdstanpy - INFO - Chain [1] start processing
20:47:04 - cmdstanpy - INFO - Chain [1] done processing
20:47:04 - cmdstanpy - INFO - Chain [1] start processing
20:47:04 - cmdstanpy - INFO - Chain [1] done processing
20:47:05 - cmdstanpy - INFO - Chain [1] start processing
20:47:05 - cmdstanpy - INFO - Chain [1]

            Actual    Predicted     Lower_CI     Upper_CI
ds                                                       
2022-09-02  625.85   696.092241 -1272.548390  2887.540701
2022-09-03    0.00 -1571.713658 -3621.432489   509.599604
2022-09-04    0.00 -1284.701003 -3192.486060   601.234172
2022-09-05    0.00  -309.032098 -2299.201255  1628.086974
2022-09-06  563.02  3593.318885  1801.479051  5668.220454


Mean Absolute Error (MAE): 643.07
Mean Squared Error (MSE): 1369079.74
Root Mean Squared Error (RMSE): 1170.08
Forecast results saved to 'Forecast_results_Prophet_ExpandingWindow.csv'.
